### Imports básicos

In [1]:
import requests
from bs4 import BeautifulSoup
import math
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from datetime import datetime
import time

In [3]:
today = datetime.today().strftime('%Y-%m-%d')

In [4]:
year = today[0:4]
month = today[5:7]
day = int(today[8:10]) -1

In [5]:
print(year)
print(month)
print(day)

2024
10
30


# Explorando os dados

## Funções para o Download das informações

In [6]:
#Função para armazenar os jogos do dia
def get_games(d, m, y, url_list):
    
    #adicionando um timer para não sobrecarregar o site
    time.sleep(random.randint(8, 12))
    
    #Definindo a url do dia
    url = f"https://www.basketball-reference.com/boxscores/?month={m}&day={d}&year={y}"
    
    #Download do site
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    #Lista de jogos do dia
    games = soup.find('div', id='content').find('div', class_='game_summaries'
                        ).find_all('div', class_='game_summary expanded nohover')
    
    #Adicionando Urls do play_by_play
    for i in games:
        url_list.append(i.find('p', class_='links').find_all('a')[1].attrs['href'])
    

In [7]:
#Função para coletar os dados de cada jogo para a tabelas de jogos
def get_game_infos(d, m, y, games_dict, url):
    
    #adicionando um timer para não sobrecarregar o site
    time.sleep(random.randint(8, 12))
    
    #Fazendo Download do site
    pbp_url = f'https://www.basketball-reference.com{url}'
    response_pbp = requests.get(pbp_url)
    soup_pbp = BeautifulSoup(response_pbp.content, "html.parser")
    
    #Getting dict keys
    dict_keys = list(games_dict)
    
    #Adicionando chave e infos do dias
    games_dict[dict_keys[0]].append(url)
    games_dict[dict_keys[1]].append(d)
    games_dict[dict_keys[2]].append(m)
    games_dict[dict_keys[3]].append(y)    
    
    #Adicionando infos do away team
    away_team_name = soup_pbp.find('div', class_="scorebox").find_all('div')[0].find('strong').find('a').text
    games_dict[dict_keys[4]].append(away_team_name)
    away_team_score = soup_pbp.find('div', class_="scorebox").find_all('div')[0].find('div', class_ = 'score').text
    games_dict[dict_keys[5]].append(away_team_score)
    away_team_key = soup_pbp.find('div', class_="scorebox").find_all('div')[0].find('strong').find('a').attrs['href']
    games_dict[dict_keys[6]].append(away_team_key)
    
    #Adicionando infos do home team
    home_team_name = soup_pbp.find('div', class_="scorebox").find_all('div')[7].find('strong').find('a').text
    games_dict[dict_keys[8]].append(home_team_name)
    home_team_score = soup_pbp.find('div', class_="scorebox").find_all('div')[7].find('div', class_ = 'score').text
    games_dict[dict_keys[9]].append(home_team_score)
    home_team_key = soup_pbp.find('div', class_="scorebox").find_all('div')[7].find('strong').find('a').attrs['href']
    games_dict[dict_keys[10]].append(home_team_key)
    
    #Calculando o Ganhador
    if int(away_team_score) > int(home_team_score):
        games_dict[dict_keys[7]].append('W')
        games_dict[dict_keys[11]].append('L')
    else:
        games_dict[dict_keys[11]].append('W')
        games_dict[dict_keys[7]].append('L')    
        
    #Coletando o play by play
    plays = soup_pbp.find('table').find_all('tr')   
    pbp = get_pbp(plays, url, away_team_name, home_team_name)
    games_dict[dict_keys[12]].append(pbp)

In [8]:
# Função para coletar o play by play
def get_pbp(pbp_table, key, at, ht):

    #Dicionários com as info básicas de cada jogo
    dict_pbp = {
    'key' : [],
    'quarter' : [],
    'time' : [],
    'score' : [],
    'team_play' : [],
    'play': [],
    }
    
    #Apoio para calcular o quarto
    q = 1
    
    #Loop para fazer o download das jogadas
    for i in pbp_table[3:]:
    
        #Checar primeiro se uma linha da tabela com jogadas
        if len(i.find_all('td')) == 6:
            dict_pbp['key'].append(key)
            dict_pbp['quarter'].append(f"Q{q}")
            dict_pbp['time'].append(i.find_all('td')[0].text)
            dict_pbp['score'].append(i.find_all('td')[3].text)
          
            #Checando se a jogada foi do time visitante ou do time da casa
            if len(i.find_all('td')[1].text) > 2:
                dict_pbp['team_play'].append(at)
                dict_pbp['play'].append(i.find_all('td')[1].text)            
            
            elif len(i.find_all('td')[5].text) > 2:
                dict_pbp['team_play'].append(ht)
                dict_pbp['play'].append(i.find_all('td')[5].text)   
            
        #Bucando as linhas da tabela que indicam a mudança de quarto
        elif len(i.find_all('th')) == 1:
            q += 1
    
    return dict_pbp
    

In [9]:
games_table = {
    'key' : [],
    'day' : [],
    'month' : [],
    'year' : [],
    'away_team' : [],
    'away_team_score' : [],
    'away_team_key' : [],
    'away_team_W/L': [],
    'home_team' : [],
    'home_team_score' : [],
    'home_team_key' : [],
    'home_team_W/L': [],
    'game_pbp' : []
}

## Checando as DFs para ver se o downaload foi eficaz

Coletando os dados

In [10]:
#Usando um dia travado para não comprometer as análises futuras
today_urls = []
get_games(d='30', m='10', y='2024', url_list=today_urls)

In [11]:
for i in today_urls:
    get_game_infos(d='30', m='10', y='2024', games_dict=games_table, url=i)
    print(i)


/boxscores/pbp/202410300CHI.html
/boxscores/pbp/202410300CHO.html
/boxscores/pbp/202410300CLE.html
/boxscores/pbp/202410300GSW.html
/boxscores/pbp/202410300IND.html
/boxscores/pbp/202410300LAC.html
/boxscores/pbp/202410300MEM.html
/boxscores/pbp/202410300MIA.html
/boxscores/pbp/202410300OKC.html
/boxscores/pbp/202410300PHI.html
/boxscores/pbp/202410300WAS.html


Checando estrutura da tabela de jogos

In [12]:
# Criando uma tabela apenas com so resultados dos jogos
table = games_table.copy()
table.pop('game_pbp')
df = pd.DataFrame(table)
df

,key,day,month,year,away_team,away_team_score,away_team_key,away_team_W/L,home_team,home_team_score,home_team_key,home_team_W/L
0,/boxscores/pbp/202410300CHI.html,30,10,2024,Orlando Magic,99,/teams/ORL/2025.html,L,Chicago Bulls,102,/teams/CHI/2025.html,W
1,/boxscores/pbp/202410300CHO.html,30,10,2024,Toronto Raptors,133,/teams/TOR/2025.html,L,Charlotte Hornets,138,/teams/CHO/2025.html,W
2,/boxscores/pbp/202410300CLE.html,30,10,2024,Los Angeles Lakers,110,/teams/LAL/2025.html,L,Cleveland Cavaliers,134,/teams/CLE/2025.html,W
3,/boxscores/pbp/202410300GSW.html,30,10,2024,New Orleans Pelicans,89,/teams/NOP/2025.html,L,Golden State Warriors,104,/teams/GSW/2025.html,W
4,/boxscores/pbp/202410300IND.html,30,10,2024,Boston Celtics,132,/teams/BOS/2025.html,L,Indiana Pacers,135,/teams/IND/2025.html,W
5,/boxscores/pbp/202410300LAC.html,30,10,2024,Portland Trail Blazers,106,/teams/POR/2025.html,W,Los Angeles Clippers,105,/teams/LAC/2025.html,L
6,/boxscores/pbp/202410300MEM.html,30,10,2024,Brooklyn Nets,119,/teams/BRK/2025.html,W,Memphis Grizzlies,106,/teams/MEM/2025.html,L
7,/boxscores/pbp/202410300MIA.html,30,10,2024,New York Knicks,116,/teams/NYK/2025.html,W,Miami Heat,107,/teams/MIA/2025.html,L
8,/boxscores/pbp/202410300OKC.html,30,10,2024,San Antonio Spurs,93,/teams/SAS/2025.html,L,Oklahoma City Thunder,105,/teams/OKC/2025.html,W
9,/boxscores/pbp/202410300PHI.html,30,10,2024,Detroit Pistons,105,/teams/DET/2025.html,W,Philadelphia 76ers,95,/teams/PHI/2025.html,L


Checando A estrutura do Play - by - Play

In [13]:
# Criando um dataframe 
df2 = pd.DataFrame(games_table['game_pbp'][0])
df2

,key,quarter,time,score,team_play,play
0,/boxscores/pbp/202410300CHI.html,Q1,11:44.0,3-0,Orlando Magic,K. Caldwell-Pope makes 3-pt jump shot from 26 ...
1,/boxscores/pbp/202410300CHI.html,Q1,11:27.0,3-2,Chicago Bulls,J. Giddey makes 2-pt jump shot from 20 ft
2,/boxscores/pbp/202410300CHI.html,Q1,11:15.0,3-2,Orlando Magic,P. Banchero misses 2-pt jump shot from 6 ft
3,/boxscores/pbp/202410300CHI.html,Q1,11:12.0,3-2,Chicago Bulls,Defensive rebound by J. Giddey
4,/boxscores/pbp/202410300CHI.html,Q1,11:01.0,3-2,Chicago Bulls,P. Williams misses 2-pt layup from 2 ft
...,...,...,...,...,...,...
454,/boxscores/pbp/202410300CHI.html,Q4,0:00.0,99-101,Chicago Bulls,Personal foul by A. Black (drawn by J. Giddey)
455,/boxscores/pbp/202410300CHI.html,Q4,0:00.0,99-101,Chicago Bulls,J. Giddey misses free throw 1 of 2
456,/boxscores/pbp/202410300CHI.html,Q4,0:00.0,99-101,Chicago Bulls,Offensive rebound by Team
457,/boxscores/pbp/202410300CHI.html,Q4,0:00.0,99-102,Chicago Bulls,J. Giddey makes free throw 2 of 2


# Explorando os tipos de jogadas para criar as colunas da nossa base de dados

### Criando lista com todas as jogadas do dia para a exploração e funções para tratar os dados

In [ ]:
#Criando uma lista com todas as jogadas do dia
plays_list = []
import string

for i in games_table['game_pbp']:
    for x in i['play']:
        plays_list.append(x)

plays_list = [x.replace('(', '').replace(')', '').replace(".",
            '').replace("..", '').replace(";", '').lower() for x in plays_list]

plays_list

In [ ]:
# Função para separar jogadas que tenham a palavra desejada
from nltk.tokenize import word_tokenize

def wanted_word_list(words_list, wanted_word):
    
    wanted_list = [i for i in words_list if wanted_word in i]
    wanted_list = [word_tokenize(i) for i in wanted_list]
    
    return wanted_list

In [ ]:
#Função para entender o número máximos de palavra de cada lista
from collections import Counter

def find_max_words(words_list):
    num_palavras = [len(sublista) for sublista in words_list]
    contagem = Counter(num_palavras)
    return max(contagem.keys())

In [ ]:
#Função para criar colunas
def columns_def(max_w):
    columns_list = []
    for i in range(1,max_w+1):
        columns_list.append(f"word{i}")
    return columns_list

### Exploranddo as palavras que mais se repetem

In [ ]:
plays_list_token = [word_tokenize(i) for i in plays_list]
plays_df = pd.DataFrame(plays_list_token)
plays_df

In [ ]:
plays_col = plays_df.columns.to_list()
fig, axs = plt.subplots(nrows=len(plays_col), ncols=1, figsize=(10, 30))

# Loop para criar os gráficos
for n, col in enumerate(plays_col):
    
    # Não precisa converter o índice 'n' em índices de linha, já que row = n
    row = n  # Cada subplot em uma nova linha
    
    top5_words = plays_df[col].value_counts(normalize=True)[0:10]
    sns.barplot(x=top5_words.index, y=top5_words.values, ax=axs[row])
    axs[row].set_title(col)  # Adicione um título ao gráfico

plt.tight_layout()  # Para melhorar a disposição dos gráficos
plt.show()


In [ ]:
top5_words = plays_df[0].value_counts(normalize=True)[0:10]
top5_words

## Colunas iniciais

Criando o dicionário que servirá de base para a base de dados tabular do play-by-play

Todas as jogadas contarão com essa informaçoes

In [ ]:
pbp_tab_data = {
    'key' : [],
    'quarter' : [],
    'time' : [],
    'team' : [],
    'player': [],
}

## Arremessos feitos

In [ ]:
makes_list = wanted_word_list(plays_list, 'makes')
max_words_make_list = find_max_words(makes_list)
columns_make_list = columns_def(max_words_make_list)

In [ ]:
makes_df = pd.DataFrame(makes_list, columns=columns_make_list)
makes_df

In [ ]:
# # Configuração das subplots

fig, axs = plt.subplots(nrows=math.ceil(len(columns_make_list)/2), ncols=2, figsize=(10, 30))

# Loop para criar os gráficos
for n, col in enumerate(columns_make_list):
    
    # Converter o índice 'n' em índices de linha e coluna
    row = n // 2
    col_idx = n % 2
    
    top5_words = makes_df[col].value_counts(normalize=True)[0:10]
    sns.barplot(x=top5_words.index, y=top5_words.values, ax=axs[row, col_idx])



### Palavras a serem adicionadas

In [ ]:
# Colunas necessárias 
pbp_tab_data['2PA'] = []
pbp_tab_data['2PM'] = []
pbp_tab_data['3PA'] = []
pbp_tab_data['3PM'] = []
pbp_tab_data['FTA'] = []
pbp_tab_data['FTM'] = []
pbp_tab_data['DUNK'] = []
pbp_tab_data['LAYUP'] = []
pbp_tab_data['HOOK'] = []
pbp_tab_data['JUMP'] = []
pbp_tab_data['shoot_dist'] = []
pbp_tab_data['AST'] = []
pbp_tab_data['AST_by'] = []
pbp_tab_data['AST_to'] = []
pbp_tab_data['technical'] = []
pbp_tab_data['flagrant'] = []

pbp_tab_data

## Arremessos perdidos

In [ ]:
misses_list = wanted_word_list(plays_list, 'misses')
max_words_miss_list = find_max_words(misses_list)
columns_misses_list = columns_def(max_words_miss_list)

In [ ]:
misses_df = pd.DataFrame(misses_list, columns=columns_misses_list)
misses_df

In [ ]:
fig, axs = plt.subplots(nrows=math.ceil(len(columns_misses_list)/2), ncols=2, figsize=(10, 30))

# Loop para criar os gráficos
for n, col in enumerate(columns_misses_list):
    
    # Converter o índice 'n' em índices de linha e coluna
    row = n // 2
    col_idx = n % 2
    
    top5_words = misses_df[col].value_counts(normalize=True)[0:10]
    sns.barplot(x=top5_words.index, y=top5_words.values, ax=axs[row, col_idx])

### Palavras a serem adicionadas

In [ ]:
# Colunas necessárias 
pbp_tab_data['BLK'] = []
pbp_tab_data['BLK_by'] = []
pbp_tab_data['BLK_to'] = []

pbp_tab_data

## Rebotes

In [ ]:
rebound_list = wanted_word_list(plays_list, 'rebound')
max_words_rebound_list = find_max_words(rebound_list)
columns_rebound_list = columns_def(max_words_rebound_list)

In [ ]:
rebound_df = pd.DataFrame(rebound_list, columns=columns_rebound_list)
rebound_df

In [ ]:
fig, axs = plt.subplots(nrows=math.ceil(len(columns_rebound_list)/2), ncols=2, figsize=(10, 30))

# Loop para criar os gráficos
for n, col in enumerate(columns_rebound_list):
    
    # Converter o índice 'n' em índices de linha e coluna
    row = n // 2
    col_idx = n % 2
    
    top5_words = rebound_df[col].value_counts(normalize=True)[0:10]
    sns.barplot(x=top5_words.index, y=top5_words.values, ax=axs[row, col_idx])

### Palavras a serem adicionadas

In [ ]:
# Colunas necessárias 
pbp_tab_data['ORB'] = []
pbp_tab_data['DRB'] = []

pbp_tab_data

## Turnover

In [ ]:
tov_list = wanted_word_list(plays_list, 'turnover')
max_words_tov_list = find_max_words(tov_list)
columns_tov_list = columns_def(max_words_tov_list)

In [ ]:
tov_df = pd.DataFrame(tov_list, columns=columns_tov_list)
tov_df

In [ ]:
fig, axs = plt.subplots(nrows=math.ceil(len(columns_tov_list)), figsize=(10, 30))

# Loop para criar os gráficos
for n, col in enumerate(columns_tov_list):
    
    # Converter o índice 'n' em índices de linha e coluna
    row = n // 1
    
    top5_words = tov_df[col].value_counts(normalize=True)[0:15]
    sns.barplot(x=top5_words.index, y=top5_words.values, ax=axs[row])
    axs[row].set_title(col)

In [ ]:
tov_df[tov_df['word5'] == 'bad']

In [ ]:
tov_df[tov_df['word5'] == 'lost']

In [ ]:
tov_df[tov_df['word5'] == 'offensive']

In [ ]:
tov_df[tov_df['word5'] == 'traveling']

In [ ]:
tov_df[tov_df['word5'] == 'back']

In [ ]:
tov_df[tov_df['word5'] == 'step']

In [ ]:
tov_df[tov_df['word5'] == 'dbl']

In [ ]:
tov_df[tov_df['word5'] == 'palming']

In [ ]:
tov_df[tov_df['word7'] == 'bounds']

In [ ]:
tov_df[tov_df['word8'] == 'bounds']

In [ ]:
tov_df[tov_df['word8'] == 'lost']

### Palavras adicionadas

In [ ]:
pbp_tab_data['TOV'] = []
pbp_tab_data['bad_pass'] = []
pbp_tab_data['lost_ball'] = []
pbp_tab_data['PF'] = []
pbp_tab_data['offensive_foul'] = []
pbp_tab_data['offensive_foul_to'] = []
pbp_tab_data['offensive_draw_by'] = []
pbp_tab_data['offensive_foul_to'] = []
pbp_tab_data['travelling'] = []
pbp_tab_data['back_court'] = []
pbp_tab_data['dbl_dribbling'] = []
pbp_tab_data['palming'] = []
pbp_tab_data['STL'] = []
pbp_tab_data['STL_by'] = []
pbp_tab_data['STL_to'] = []

pbp_tab_data

## Foul

In [ ]:
foul_list = wanted_word_list(plays_list, 'foul')
max_words_foul_list = find_max_words(foul_list)
columns_foul_list = columns_def(max_words_foul_list)

In [ ]:
foul_df = pd.DataFrame(foul_list, columns=columns_foul_list)
foul_df

In [ ]:
fig, axs = plt.subplots(nrows=math.ceil(len(columns_foul_list)), figsize=(10, 30))

# Loop para criar os gráficos
for n, col in enumerate(columns_foul_list):
    
    # Converter o índice 'n' em índices de linha e coluna
    row = n // 1
    
    top5_words = foul_df[col].value_counts(normalize=True)[0:15]
    sns.barplot(x=top5_words.index, y=top5_words.values, ax=axs[row])
    axs[row].set_title(col)

In [ ]:
foul_df[foul_df['word1'] == 'shooting']

In [ ]:
foul_df[foul_df['word1'] == 'personal']

In [ ]:
foul_df[foul_df['word1'] == 'offensive']

In [ ]:
foul_df[foul_df['word1'] == 'loose']

In [ ]:
foul_df[foul_df['word1'] == 'technical']

In [ ]:
foul_df[foul_df['word1'] == 'flagrant']

In [ ]:
foul_df[foul_df['word1'] == 'def']

In [ ]:
foul_df[foul_df['word1'] == 'away']

In [ ]:
! git add.

! git commit -m "31/10/2024"

In [ ]:
list_test = ['PTS', '2PA', '3PM']

In [ ]:
for i in list(teste_dict.keys()):
    if i not in list_test:
        teste_dict[i].append(0)

In [ ]:
def complete_row(sd, rows):
    
    for i in list(sd.keys()):
        if i not in rows:
            sd[i].append(0)

In [ ]:
complete_row(teste_dict, list_test)